<a href="https://colab.research.google.com/github/vincent4u/CE807_Text_Analytics/blob/main/Model_Dis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import numpy as np
import os
import pickle
import pandas as pd

In [26]:
student_id = 2314555
np.random.seed(student_id)

In [ ]:
train_df = pd.read_csv('/content/train.csv')
#train_df.head(20)

In [ ]:
val_df=pd.read_csv('/content/valid.csv')
#val_df.head(20)

In [ ]:
test_df=pd.read_csv('/content/test.csv')
test_df.head(20)

In [35]:
print(train_df.columns)

Index(['comment_id', 'comment', 'split', 'toxicity'], dtype='object')


In [ ]:
print(test_df.columns)

In [44]:
nltk.download('punkt')
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [48]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


# Preprocessing steps
def preprocess_text(text):
    # Remove special tokens
    text = re.sub(r'SDATA_\d+\s:\s|EDATA_\d+', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove extra white spaces
    text = re.sub(r'\s+', ' ', text)

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming or lemmatization
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Preprocess the text
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

# Define the pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression())
])

# Define the parameter grid for grid search
param_grid = {
    'vectorizer__max_features': [1000, 5000, 10000],
    'classifier__C': [0.1, 1, 10],
    'classifier__penalty': ['l1', 'l2']
}

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5)
grid_search.fit(train_df['preprocessed_comment'], train_df['toxicity'])

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Testing the best model on validation data
val_pred = best_model.predict(val_df['preprocessed_comment'])

# Calculate metrics
accuracy = accuracy_score(val_df['toxicity'], val_pred)
recall = recall_score(val_df['toxicity'], val_pred, average='macro')
precision = precision_score(val_df['toxicity'], val_pred, average='macro')
f1 = f1_score(val_df['toxicity'], val_pred, average='macro')

# Print metrics
print('Accuracy:', accuracy)
print('Recall (macro):', recall)
print('Precision (macro):', precision)
print('F1 Score (macro):', f1)

# Compute and print confusion matrix
labels = np.unique(np.concatenate((val_df['toxicity'], val_pred)))
cm = confusion_matrix(val_df['toxicity'], val_pred, labels=labels)
print('Confusion Matrix:')
print(cm)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy: 0.8609589041095891
Recall (macro): 0.49879228668963055
Precision (macro): 0.4860029341953029
F1 Score (macro): 0.4698828798534746
Confusion Matrix:
[[2511   26]
 [ 380    3]]


In [47]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(max_iter=100))
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'vectorizer__max_features': [1000, 5000, 10000],
    'classifier__C': [0.1, 1.0, 10.0]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5)

# Perform hyperparameter tuning using GridSearchCV
grid_search.fit(train_df['comment'], train_df['toxicity'])

# Get the best model and its associated hyperparameters
best_model = grid_search.best_estimator_

# Testing the best model on the validation data
val_pred = best_model.predict(val_df['comment'])

accuracy = accuracy_score(val_df['toxicity'], val_pred)
recall = recall_score(val_df['toxicity'], val_pred, average='macro')
precision = precision_score(val_df['toxicity'], val_pred, average='macro')
f1 = f1_score(val_df['toxicity'], val_pred, average='macro')

# Print metrics
print('Accuracy:', accuracy)
print('Recall (macro):', recall)
print('Precision (macro):', precision)
print('F1 Score (macro):', f1)

# Compute and print confusion matrix
labels = np.unique(np.concatenate((val_df['toxicity'], val_pred)))
cm = confusion_matrix(val_df['toxicity'], val_pred, labels=labels)
print('Confusion Matrix:')
print(cm)

# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_
print('Best Hyperparameters:', best_params)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy: 0.8602739726027397
Recall (macro): 0.4961813206321893
Precision (macro): 0.45249708747807604
F1 Score (macro): 0.4648699636167632
Confusion Matrix:
[[2511   26]
 [ 382    1]]
Best Hyperparameters: {'classifier__C': 0.1, 'vectorizer__max_features': 10000}


Scoring